# Positional encoding and other functions

In [1]:
import numpy as np  
import re
import cupy as cp
import pickle
import time  
import pandas as pd
import numpy as np
import jax
import spacy
from tqdm import tqdm
from pathlib import Path
import json

cp.set_printoptions(edgeitems=30, linewidth=100000, formatter=dict(float=lambda x: "%.3g" % x)) 


def log_time(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()  # Record start time
        result = func(*args, **kwargs)  # Execute the wrapped function
        end_time = time.time()  # Record end time
        elapsed_time = end_time - start_time
        print(f"Function '{func.__name__}' executed in {elapsed_time:.4f} seconds")
        return result

    return wrapper


def get_positional_encoding(seq_len, d_model):
    
    """
    Returns a non-learnable (sinusoidal) positional encoding.
    
    
    seq_len: Length of the input sequence.
    d_model: Dimension of the embeddings.
    """
    pos = cp.arange(seq_len)[:, cp.newaxis]  # Shape: [seq_len, 1]
    i = cp.arange(d_model)[cp.newaxis, :]    # Shape: [1, d_model]

    angle_rates = 1 / cp.power(10000, (2 * (i // 2)) / cp.float32(d_model))

    # Apply sine to even indices, cosine to odd indices
    pos_encoding = cp.zeros((seq_len, d_model))
    pos_encoding[:, 0::2] = cp.sin(pos * angle_rates[:, 0::2])  # sine on even indices
    pos_encoding[:, 1::2] = cp.cos(pos * angle_rates[:, 1::2])  # cosine on odd indices

    return pos_encoding

def softmax(x, axis=-1):
    # Subtract the max value for numerical stability
    e_x = cp.exp(x - cp.max(x, axis=axis, keepdims=True))
    return e_x / cp.sum(e_x, axis=axis, keepdims=True)
def layer_norm(x, epsilon=1e-6):
    # Calculate the mean and variance
        mean = cp.mean(x, axis=-1, keepdims=True)
        var = cp.var(x, axis=-1, keepdims=True) 
        
        # Normalize the output
        x_norm = (x - mean) / cp.sqrt(var + epsilon) 
        #print(x)
        #print(mean)
        #print("mean",mean.shape)
        #print("x_norm.shape",x_norm.shape)
        return x_norm,mean,var,x.shape[-1]
def relu(x):
    return cp.maximum(0, x)
#@log_time
def pad_sequence(seq, max_len, pad_value=0):
    """Pad a sequence with a given value up to max_len."""
    current_len = seq.shape[0]
    pad_width = max_len - current_len
    if pad_width > 0:
        # Pad sequence with zeros (or any pad_value you provide)
        seq = cp.pad(seq, ((0, pad_width), (0, 0)), mode='constant', constant_values=pad_value)
    return seq
@log_time
def create_timestaped_input(input_d,words_per_phrase):
    input_translation=[]
    for j in range(input_d.shape[0]):
    # Create padded sequences
        padded_sequences = [pad_sequence(input_d[j][0:i], words_per_phrase) for i in range(1, input_d.shape[1] + 1)]
        input_translation.append(padded_sequences)
    return cp.array(input_translation)

def cross_entropy_loss(predictions, target):
    # Cross-entropy loss for a batch of predictions and targets
    batch_loss = -cp.sum(target * cp.log(predictions + 1e-9), axis=1)
    return cp.mean(batch_loss)

def diff_norm(X,var,mu,N):
    epsilon=1e-6
    AA=((1-(1/N))*(1/(cp.sqrt(var+epsilon))))
    BB=(1/N)*((X-mu)**2)
    CC=((var+epsilon)**(3/2))
    result=(AA-(BB/CC)) 
    return result

def redimension(X):
    return cp.concatenate(cp.swapaxes(X,0,1),axis=-1) 

def diffQKV(dAttention,Attention_weights,X1,X2,X3,dk):
    dAttention_weights=Attention_weights*(1-Attention_weights)
    V1=redimension(dAttention_weights@X1/cp.sqrt(dk)) 
    
    V2=redimension(X2)
    
    V3=V1*V2*X3
    dLoss_dX=cp.sum(cp.transpose(dAttention,(0,2,1))@V3,axis=0)
    return dLoss_dX



@log_time
def create_vocabulary(complete_text,name,nlp): 
        # Use re.findall to split considering punctuation
        text = re.findall(r'\[.*?\]|\w+|[^\w\s]', complete_text)
        words_list = list(set(text))
        vocabulary=dict()
        for i,j in enumerate(words_list):
            #vocabulary[j]=(jax.random.uniform(jax.random.key(cp.random.randint(10000)),embedding_size),i)
            vocabulary[j]=(cp.array(nlp(j).vector),i)
            #print(j,len(cp.array(nlp(j).vector)))

        #print(vocabulary)
        #print("Vocabulary size: ", len(vocabulary))
        with open(f"data/{name}.pkl", 'wb') as handle:
            pickle.dump(vocabulary, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
        return vocabulary



@log_time
def pad_sequences(sentences,lenght, pad_token='[PAD]',target_type=None):
        """
        Pads the input sentences to have the same length by adding [PAD] tokens at the end.
        """
        
        if target_type=="encoder": 
            # Split each sentence into words
            tokenized_sentences = [ ["[START]"] + re.findall(r'\[.*?\]|\w+|[^\w\s]',sentence)+ ["[END]"] for sentence in sentences]
        elif target_type=="decoder": 
             tokenized_sentences = [ ["[START]"] + re.findall(r'\[.*?\]|\w+|[^\w\s]',sentence) for sentence in sentences]
        elif target_type=="target": 
             tokenized_sentences = [ re.findall(r'\[.*?\]|\w+|[^\w\s]',sentence) + ["[END]"]  for sentence in sentences]
        #print(tokenized_sentences)
        if lenght==0: 
            # Find the maximum sentence length
            max_len = max(len(sentence) for sentence in tokenized_sentences)
        else:
            max_len=lenght
        
        # Pad each sentence with the [PAD] token to make them of equal length
        padded_sentences = [" ".join(sentence + [pad_token] * (max_len - len(sentence))) for sentence in tokenized_sentences]
        
        return padded_sentences

@log_time
def generate_input_encoder(x_batch,vocabulary_encoder,max_words_per_phrase): 
        
        xi=[]
      
        phrase_vectors_x = [re.findall(r'\[.*?\]|\w+|[^\w\s]', x) for x in x_batch]
      
        phrase_vectors_x= [i[0:max_words_per_phrase] for i in phrase_vectors_x] 

        #print("phrase_vectors_x:\n",phrase_vectors_x)
         
        xi=cp.array([[vocabulary_encoder[word][0] for word in phrase_vector] for phrase_vector in phrase_vectors_x]) 
        
        return xi 
    


 


@log_time
def create_input_encoder(X,vocabulary_encoder,max_words_per_phrase,embedding_size): 
    pos_encoding=get_positional_encoding(max_words_per_phrase,embedding_size)
    x_train=pad_sequences(X,max_words_per_phrase,target_type="encoder")
    
    #vocabulary_encoder["[PAD]"]=(cp.zeros(embedding_size),vocabulary_encoder["[PAD]"][1])
    inputs_e=generate_input_encoder(x_train,vocabulary_encoder,max_words_per_phrase)
    inputs_e+=pos_encoding
    return inputs_e
        
@log_time
def create_decoder_input(y_train,embedding_size,max_words_per_phrase,vocabulary_decoder):
    
     
    
    #vocabulary_decoder["[PAD]"]=(cp.zeros(embedding_size),vocabulary_decoder["[PAD]"][1])

    decoder_input=pad_sequences(y_train,lenght=max_words_per_phrase,target_type="decoder")
    decoder_input=[i.split() for i in decoder_input]
    #print(max_words_per_phrase)
    if max_words_per_phrase==None:
        max_words_per_phrase=len(decoder_input[0])

    
    phrase_vectors_y= [i[0:max_words_per_phrase] for i in decoder_input]
    # for sentence in phrase_vectors_y:
    #     print(sentence)
    yi=cp.array([ [ vocabulary_decoder[word][0]  for word in phrase_vector ] for phrase_vector in phrase_vectors_y])
    pos_encoding=get_positional_encoding(max_words_per_phrase,embedding_size)
    #print(pos_encoding.shape,yi.shape)
    yi=yi+pos_encoding
    decoder_inputs=cp.swapaxes(create_timestaped_input(yi,max_words_per_phrase),0,1)
    return decoder_inputs
#@log_time
def get_one_hot(word,vocabulary_decoder): 
    #print(word)
    vocab_size=len(vocabulary_decoder)
    one_hot_vector = cp.zeros(vocab_size)
    one_hot_vector[vocabulary_decoder[word][1]] = vocabulary_decoder[word][1]
    return one_hot_vector

@log_time
def create_target(y_train,max_words_per_phrase,vocabulary_decoder):
    target_d=pad_sequences(y_train,lenght=max_words_per_phrase,target_type="target")
    target_d=[i.split() for i in target_d]
    target_d=[[get_one_hot(index, vocabulary_decoder) for index in phrase] for phrase in target_d]
    target_d=cp.swapaxes(create_timestaped_input(cp.array(target_d),max_words_per_phrase),0,1) 
    targets_d=[]

    for i in range(target_d.shape[0]):
        ff=[]
        #print(i,target_d[i].shape)
        for j in range(target_d[i].shape[0]):
            ff.append(target_d[i][j][i])
            #print(targets_d[i][j][i])
        targets_d.append(ff)
            #print(ff) 
    targets_d=cp.array(targets_d)
    return targets_d








# Summary Encoder

In [2]:
 
def forward_attention_encoder(inputs_e):
    global Qe,Ke,Ve,num_heads,batch_size,dk
 
    Q_E= cp.swapaxes(cp.array(cp.array_split(cp.matmul(inputs_e, Qe),num_heads,axis=2)), 0, 1)
    #print("Qval.shape: ",Q_E.shape)

    K_E = cp.swapaxes(cp.array(cp.array_split(cp.matmul(inputs_e, Ke),num_heads,axis=2)), 0, 1)
    #print("Kval.shape: ",K_E.shape)


    V_E = cp.swapaxes(cp.array(cp.array_split(cp.matmul(inputs_e,Ve),num_heads,axis=2)), 0, 1)
    #print("Vval.shape: ",V_E.shape)

    QKscaled = cp.matmul(Q_E, cp.transpose(K_E, (0, 1, 3, 2))) / cp.sqrt(dk)

    Attention_weights_e = softmax(QKscaled)
    #print("Attention_weights shape:",Attention_weights_e.shape)


    Ae = cp.matmul(Attention_weights_e, V_E)
    #print("Attention shape:",Ae.shape)


    Ae=cp.array([cp.concatenate(Ae[i], axis=1) for i in range(batch_size)])
    #print("Attention shape concat:",Ae.shape)

    Xe=Ae+inputs_e
    Ect1,mu_e,var_e,Ne=layer_norm(Xe)
    #print("Ect1.shape",Ect1.shape,Ne)

    return Ae,Xe,Ect1,mu_e,var_e,Ne,Attention_weights_e,K_E,V_E,Q_E


def fully_connected_layers_encoder(Ect1):
    
    global Wfl1e,bfl1e,Wfl2e,bfl2e
    
    Xe1=cp.matmul(Ect1,Wfl1e)+bfl1e
    FLe1=relu(Xe1)

    FLe2=cp.matmul(FLe1,Wfl2e)+bfl2e

    Xe2=FLe2+Ect1
    Ecout,mu_e2,var_e2,N_e2=layer_norm(Xe2)
    #print("Ecout.shape",Ecout.shape)
    return Ecout,mu_e2,var_e2,N_e2,FLe1,Xe1,Xe2





In [3]:
def cross_attention_encoder(Ecout):
    global Kc,Vc 
    K_C  = cp.swapaxes(cp.array(cp.array_split(cp.matmul(Ecout, Kc),num_heads,axis=2)), 0, 1)
    #print("K_C.shape: ",K_C.shape)# shape is: num_phrases, numbheads, words_per_phrase, dv/num_heads 
    V_C  = cp.swapaxes(cp.array(cp.array_split(cp.matmul(Ecout,Vc),num_heads,axis=2)), 0, 1)
    #print("V_C.shape: ",V_C.shape)
    return K_C,V_C




## Decoder forward

In [4]:
def forward_attention_decoder(input_decoder):
    global Qd,Kd,Vd,words_per_phrase
    Q_D  = cp.swapaxes(cp.array(cp.array_split(cp.matmul(input_decoder, Qd),num_heads,axis=2)), 0, 1)
    #print("Qval.shape: ",Q_D.shape)# numwords, num_phrases, numheads, num_words, dv/num_heads

    #K_D  = cp.swapaxes(cp.swapaxes(cp.array(cp.array_split(cp.matmul(inputs_d[step], Kd),num_heads,axis=3)), 0, 1),1,2)
    K_D  = cp.swapaxes(cp.array(cp.array_split(cp.matmul(input_decoder, Kd),num_heads,axis=2)), 0, 1)
    #print("Kval.shape: ",K_D.shape)


    #V_D  = cp.swapaxes(cp.swapaxes(cp.array(cp.array_split(cp.matmul(inputs_d[step], Vd),num_heads,axis=3)), 0, 1),1,2)
    V_D  = cp.swapaxes(cp.array(cp.array_split(cp.matmul(input_decoder, Vd),num_heads,axis=2)), 0, 1)

    QKscaled_decoder  = cp.matmul(Q_D, cp.transpose(K_D, (0, 1, 3, 2))) / cp.sqrt(dv)
    # Step 1: Create a causal mask of shape (1, 1, 9, 9) to broadcast across heads and batch
    mask_size=words_per_phrase
    mask = cp.tril(cp.ones((mask_size, mask_size)))  # (9, 9) lower triangular matrix
    mask[mask == 0]=-cp.inf  # Set future tokens to -inf
    mask[mask == 1]=0  # Set allowed tokens to 0
    mask = mask.reshape(1, 1, mask_size, mask_size)   

    # Step 2: Apply mask to QKscaled_decoder (it will broadcast across batch and heads)
    QKscaled_decoder = QKscaled_decoder + mask 

    Attention_weights_masked = softmax(QKscaled_decoder)


    A_mask = cp.matmul(Attention_weights_masked, V_D)
    #print("A_mask.shape non concat: ",A_mask.shape)
    
    #A_mask=cp.swapaxes(cp.concatenate(cp.swapaxes(A_mask,0,2),axis=-1),0,1)
    A_mask= cp.concatenate(cp.swapaxes(A_mask,0,1),axis=-1) 

    Xd = input_decoder + A_mask
    Dt1,mu_d,var_d,N_d = layer_norm(Xd)
    #print("A_mask.shape concat: ",A_mask.shape)
    #print("inputs_d.shape: ",input_decoder.shape)
    #print("Dt1.shape: ",Dt1.shape)
    return A_mask,Xd,Dt1,mu_d,var_d,N_d,Attention_weights_masked,Q_D,K_D,V_D
    
def cross_attention_decoder(Dt1):
    global Qc 
    Q_C  = cp.swapaxes(cp.array(cp.array_split(cp.matmul(Dt1, Qc),num_heads,axis=2)), 0, 1)
    #print("Q_C.shape: ",Q_C.shape)
    return Q_C

def cross_attention(Q_C,K_C,V_C,Dt1):
    global dv
    QKscaled_cross_attention  = cp.matmul(Q_C, cp.transpose(K_C , (0, 1, 3, 2)))/ cp.sqrt(dv)
    Attention_weights_cross = softmax(QKscaled_cross_attention)
    Acr = cp.matmul(Attention_weights_cross, V_C)
    #print("Acr.shape non concat",Acr.shape)
    Acr=cp.concatenate(cp.swapaxes(Acr,0,1),axis=-1)
    #print("Acr.shape concat",Acr.shape)
    Res=Acr + Dt1
    Dt2, mu_res,var_res,N_res = layer_norm(Res)
    return Dt2, mu_res,var_res,N_res,Res,Attention_weights_cross

def fully_connected_layers_decoder(Dt2):
    
    global Wfl1d,bfl1d,Wfl2d,bfl2d,num_phrases
     
    Xd1=cp.matmul(Dt2,Wfl1d)+bfl1d
   
    FLd1=relu(Xd1)
     
    FLd2=cp.matmul(FLd1,Wfl2d)+bfl2d
    #print("FLd2.shape",FLd2.shape)
  
    Xd2=FLd2+Dt2
    Dout,mu_d2,var_d2,N_d2=layer_norm(Xd2)
 
    #print("Dout.shape",Dout.shape)
    Dout=Dout.reshape(Dout.shape[0],Dout.shape[1]*Dout.shape[2])
    #print("Dout.shape",Dout.shape)
    return Dout,mu_d2,var_d2,N_d2,Xd2,Xd1,FLd1

def output_layer(Dout):
    
    global Wo,bo 
    #print(Dout.shape,W0.shape)  
    Zout=cp.matmul(Dout,Wo)+bo 
    SigmaZout = softmax(Zout) 
    #print("SigmaZout.shape",SigmaZout.shape)
    
    return SigmaZout 


def loss_calculation(SigmaZout,target):
    #print("target.shape",cp.array(target).shape)
    Loss=cross_entropy_loss(SigmaZout, target)
    #print("Loss:",Loss)
    return Loss




## BackPropagation

In [5]:
def derivate_dout(SigmaZout,target,Dout):
    global Wo,embedding_size,batch_size,words_per_phrase
    dLoss_dZout=SigmaZout-target
    #print("dLoss_dZout.shape",dLoss_dZout.shape)
    dLoss_W0=cp.transpose(dLoss_dZout,(1,0))@Dout
    #print("dLoss_W0.shape",dLoss_W0.shape,"W0.shape",W0.shape)
    dLoss_b0=cp.sum(dLoss_dZout, axis=0)
    #print("dLoss_b0.shape",dLoss_b0.shape,"b0.shape",b0.shape)
    dLoss_Dout=dLoss_dZout@Wo.T
    dLoss_Dout=dLoss_Dout.reshape(batch_size,words_per_phrase,embedding_size)
    #print("dLoss_Dout.shape",dLoss_Dout.shape)
    return dLoss_Dout,dLoss_W0,dLoss_b0


def derivate_fully_connected_layers_decoder(dLoss_Dout,Dt2,Xd2,var_d2,mu_d2,N_d2,Wfl2d,FLd1,Xd1):
 
    dLoss_FLd2=dLoss_Dout*diff_norm(Xd2,var_d2,mu_d2,N_d2)
    #print("dLoss_FLd2.shape",dLoss_FLd2.shape) 
    dLoss_Dt2_a=dLoss_FLd2
    #print("dLoss_Dt2_a.shape",dLoss_Dt2_a.shape) 
    #print("Dt2.shape",Dt2.shape) 
    dLoss_FLd1=dLoss_FLd2@cp.transpose(Wfl2d,(1,0))
    #print("dLoss_FLd1.shape",dLoss_FLd1.shape) 
    #print("FLd1.shape",FLd1.shape) 
    dLoss_Wfl2d=cp.sum(cp.transpose(dLoss_FLd2,(0,2,1))@FLd1,axis=0)
    #print("dLoss_Wfl2d.shape",dLoss_Wfl2d.shape) # do the mean here over each phrase
    #print("Wfl2d.shape",Wfl2d.shape) 
    dLoss_bfl2d=cp.sum(cp.sum(dLoss_FLd2, axis=0),axis=0)
    #print("dLoss_bfl2d.shape",dLoss_bfl2d.shape) # do the mean here over each phrase
    #print("bfl2d.shape",bfl2d.shape) 
    if Xd1.all()>0:
        DLoss_Dt2_b=dLoss_FLd1@cp.transpose(Wfl1d,(1,0))
    else:
        DLoss_Dt2_b=0
    DLoss_Dt2=dLoss_Dt2_a+DLoss_Dt2_b
    #print("DLoss_Dt2.shape",DLoss_Dt2.shape) # do the mean here over each phrase
    #print("Dt2.shape",Dt2.shape) 
    if Xd1.all()>0:
        dLoss_Wfl1d=cp.sum(cp.transpose(dLoss_FLd1,(0,2,1))@Dt2,axis=0)
    else:
        dLoss_Wfl1d=0
    #print("dLoss_Wfl1d.shape",dLoss_Wfl1d.shape) # do the mean here over each phrase
    #print("Wfl1d.shape",Wfl1d.shape) 
    if Xd1.all()>0:
        dLoss_bfl1d=cp.sum(cp.sum(dLoss_FLd1,axis=0),axis=0)
    else:
        dLoss_bfl1d=0
    return dLoss_Wfl2d,dLoss_bfl2d,dLoss_Wfl1d,dLoss_bfl1d,DLoss_Dt2

def derivative_cross_attention(dLoss_Dt2,Res,var_res,mu_res,N_res,Attention_weights_cross,K_C,V_C,Q_C,Ecout,Dt1):
    #print("dLoss_bfl1d.shape",dLoss_bfl1d.shape) # do the mean here over each phrase
    #print("bfl1d.shape",bfl1d.shape) 
    
    dLoss_Acr=dLoss_Dt2*diff_norm(Res,var_res,mu_res,N_res)
    #print("dLoss_Acr.shape",dLoss_Acr.shape) # do the mean here over each phrase
    #print("Acr.shape",Acr.shape) 
    dLoss_Dt1_a=dLoss_Dt2*diff_norm(Res,var_res,mu_res,N_res)
    #print("dLoss_Dt1.shape",dLoss_Dt1_a.shape) # do the mean here over each phrase
    #print("Dt1.shape",Dt1.shape) 
    dLoss_Qc=diffQKV(dLoss_Acr,Attention_weights_cross,K_C,V_C,Dt1,dk)
    #print("dLoss_dQc.shape",dLoss_Qc.shape) # do the mean here over each phrase
    #print("Qc.shape",Qc.shape) 
    dLoss_Kc=diffQKV(dLoss_Acr,Attention_weights_cross,Q_C,V_C,Ecout,dk)
    #print("dLoss_dKc.shape",dLoss_Kc.shape) # do the mean here over each phrase
    #print("Kc.shape",Kc.shape) 
    dLoss_Vc=f=cp.sum(cp.mean(cp.transpose(cp.expand_dims(dLoss_Acr, axis=1),(0,1,3,2))@(Attention_weights_cross@cp.expand_dims(Ecout, axis=1)),axis=1),axis=0)
    #print("dLoss_dVc.shape",dLoss_Vc.shape) # do the mean here over each phrase
    #print("Vc.shape",Vc.shape) 
    return dLoss_Qc,dLoss_Kc,dLoss_Vc,Attention_weights_cross,dLoss_Dt1_a,dLoss_Acr
    
def derivative_attention_decoder(dLoss_Acr,Attention_weights_cross,dLoss_Dt1_a,Attention_weights_masked,Q_D,V_D,K_D,K_C,V_C,Xd,var_d,mu_d,N_d,input_d):
    global Qc
    dAttention_weights_cross=Attention_weights_cross*(1-Attention_weights_cross)
    V1=redimension(dAttention_weights_cross@K_C/cp.sqrt(dk)) 

    V2=redimension(V_C)

    V3=V1*V2@Qc
    dLoss_Dt1_b=dLoss_Acr*V3
    #print("dLoss_Dt1_b.shape",dLoss_Dt1_b.shape) # do the mean here over each phrase
    #print("dLoss_Dt1_a.shape",dLoss_Dt1_a.shape) 
    dLoss_Dt1=dLoss_Dt1_a+dLoss_Dt1_b
    dLoss_Amask=dLoss_Dt1*diff_norm(Xd,var_d,mu_d,N_d)
    #print("dLoss_DAmask.shape",dLoss_Amask.shape)  
    dLoss_inputd_a=dLoss_Amask
    #print("dLoss_Dinputd_a.shape",dLoss_inputd_a.shape) 
    dLoss_Kd=diffQKV(dLoss_Amask,Attention_weights_masked,Q_D,V_D,input_d,dk) 
    #print("dLoss_Kd.shape",dLoss_Kd.shape) 
    dLoss_Qd=diffQKV(dLoss_Amask,Attention_weights_masked,K_D,V_D,input_d,dk) 
    #print("dLoss_Qd.shape",dLoss_Qd.shape) 
    dLoss_Vd=f=cp.sum(cp.mean(cp.transpose(cp.expand_dims(dLoss_Amask, axis=1),(0,1,3,2))@(Attention_weights_masked@cp.expand_dims(input_d, axis=1)),axis=1),axis=0)
    return dLoss_Kd,dLoss_Qd,dLoss_Vd,dLoss_inputd_a,dLoss_Amask


def derivative_input_decoder(dLoss_Amask,Attention_weights_masked,K_D,V_D,Q_D,dLoss_inputd_a,input_d):
    global Qd,Kd,Vd
    dLoss_V_D=cp.transpose(cp.mean(cp.transpose(cp.expand_dims(dLoss_Amask, axis=1),(0,1,3,2))@Attention_weights_masked,axis=1),(0,2,1))
    dLoss_V_D.shape
    dLoss_inputd_v=dLoss_V_D@Vd

    # print("dLoss_inputd_v.shape",dLoss_inputd_v.shape) # do the mean here over each phrase
    # print("input_d.shape",input_d.shape) 

    dAttention_weights_masked=Attention_weights_masked*(1-Attention_weights_masked)
    V1=redimension(dAttention_weights_masked@K_D/cp.sqrt(dk)) 
    V2=redimension(V_D)
    V3=V1*V2
    dLoss_Q_D=dLoss_Amask*V3
    dLoss_Q_D.shape
    dLoss_inputd_q=dLoss_Q_D@Qd
    #print("dLoss_inputd_q.shape",dLoss_inputd_q.shape)
 
    V1=redimension(dAttention_weights_masked@Q_D/cp.sqrt(dk)) 
    V2=redimension(V_D)
    V3=V1*V2
    dLoss_K_D=dLoss_Amask*V3 
    dLoss_inputd_k=dLoss_K_D@Kd
    #print("dLoss_inputd_k.shape",dLoss_inputd_k.shape)
    dLoss_inputd=dLoss_inputd_a+dLoss_inputd_k+dLoss_inputd_q+dLoss_inputd_v

    dLoss_dWemb_decoder=dLoss_inputd*input_d
    return dLoss_inputd,dLoss_dWemb_decoder
    




In [6]:
def derivative_Ecout(Attention_weights_cross,dLoss_Acr,Q_C,V_C):
    global Kc,Vc
    dAttention_weights_cross=Attention_weights_cross*(1-Attention_weights_cross)
    V1=redimension(dAttention_weights_cross@Q_C/cp.sqrt(dk)) 

    V2=redimension(V_C)

    V3=V1*V2

    
    dLoss_K_C=dLoss_Acr*V3
    dLoss_K_C.shape

    dLoss_Ecout_k=dLoss_K_C@Kc
    #print("dLoss_Ecout_k.shape",dLoss_Ecout_k.shape) 

    dLoss_V_C=cp.transpose(cp.mean(cp.transpose(cp.expand_dims(dLoss_Acr, axis=1),(0,1,3,2))@Attention_weights_cross,axis=1),(0,2,1))
    dLoss_V_C.shape
    dLoss_Ecout_v=dLoss_V_C@Vc

    #print("dLoss_Ecout_v.shape",dLoss_Ecout_v.shape) # do the mean here over each phrase
    dLoss_Ecout=dLoss_Ecout_k+dLoss_Ecout_v
    return dLoss_Ecout

def derivate_fully_connected_layers_encoder(dLoss_Ecout,Ect1,Xe2,var_e2,mu_e2,N_e2,FLe1,Xe1):
    global Wfl2e,Wfl1e
    dLoss_dFLe2=dLoss_Ecout*diff_norm(Xe2,var_e2,mu_e2,N_e2)
    dLoss_Ect1_a=dLoss_dFLe2
    #print(Wfl2e.shape)
    dLoss_dFLe1=dLoss_dFLe2@cp.transpose(Wfl2e,(1,0))
    dLoss_dWfl2e=cp.transpose(dLoss_dFLe2,(0,2,1))@FLe1
    #print(dLoss_dWfl2e)
    dLoss_dbfl2e=cp.sum(dLoss_dFLe2,axis=1) 
    if Xe1.all()>0:
        dLoss_Ect1_b=dLoss_dFLe1@cp.transpose(Wfl1e,(1,0))
    else:
        dLoss_Ect1_b=0

    dLoss_Ect1=dLoss_Ect1_b+dLoss_Ect1_a
    if Xe1.all()>0:
        dLoss_Wfl1e=cp.transpose(dLoss_dFLe1,(0,2,1))@Ect1
    else:
        dLoss_Wfl1e=0
 
    if Xe1.all()>0:
        dLoss_bfl1e=cp.transpose(dLoss_dFLe1,(0,2,1)) 
    else:
        dLoss_bfl1e=0
 
    return dLoss_dWfl2e,dLoss_dbfl2e,dLoss_Wfl1e,dLoss_bfl1e,dLoss_Ect1
import warnings
 
def derivative_attention_encoder(dLoss_Ect1,Xe,var_e,mu_e,Ne,Attention_weights_e,K_E,V_E,Q_E):
    global inputs_e
    with warnings.catch_warnings():
        warnings.simplefilter("error", RuntimeWarning)
        try:
            dLoss_Ae=dLoss_Ect1*diff_norm(Xe,var_e,mu_e,Ne)
            
            dLoss_inpute_a=dLoss_Ae
            
            dLoss_dQe=diffQKV(dLoss_Ae,Attention_weights_e,K_E,V_E,inputs_e,dk)
            #print("dLoss_dQe.shape",dLoss_dQe.shape) # do the mean here over each phrase
            #print("Qe.shape",Qe.shape) 
            dLoss_dKe=diffQKV(dLoss_Ae,Attention_weights_e,Q_E,V_E,inputs_e,dk)
            #print("dLoss_dKe.shape",dLoss_dKe.shape) # do the mean here over each phrase
            #print("Ke.shape",Ke.shape) 
            
           
            dLoss_dVe=cp.sum(cp.sum(cp.transpose(cp.expand_dims(dLoss_Ae, axis=1),(0,1,3,2))@(Attention_weights_e@cp.expand_dims(inputs_e, axis=1)),axis=1),axis=0)
            #print("dLoss_dVe.shape",dLoss_dVe.shape) # do the mean here over each phrase
            return dLoss_dQe,dLoss_dKe,dLoss_dVe,dLoss_inpute_a,dLoss_Ae
        except RuntimeWarning as rw:
            # Check for NaN or inf values in inputs and matrices
            print("dLoss_Ae check ", dLoss_Ae)
            print("Attention_weights_e  ", Attention_weights_e)
            print("inputs_e  ", inputs_e)
            print(f"Caught a RuntimeWarning: {rw}")
            return None  # Return None if a warning occurs
        
        except Exception as e:
            # Additional checks in case of other exceptions
            print("inputs_e check ", cp.isnan(inputs_e).any(), cp.isinf(inputs_e).any())
            print("Attention_weights_e check ", cp.isnan(Attention_weights_e).any(), cp.isinf(Attention_weights_e).any())
            print("dLoss_Ae check ", cp.isnan(dLoss_Ae).any(), cp.isinf(dLoss_Ae).any())
            print(f"Caught an error: {e}")
            return None 


def derivative_input_encoder(dLoss_Ae,Attention_weights_e,K_E,V_E,Q_E,dLoss_inpute_a):
    global Ve,Qe,Ke,inputs_e
    dLoss_V_E=cp.transpose(cp.sum(cp.transpose(cp.expand_dims(dLoss_Ae, axis=1),(0,1,3,2))@Attention_weights_e,axis=1),(0,2,1))
    dLoss_inpute_v=dLoss_V_E@Ve

  
    dAttention_weights_e=Attention_weights_e*(1-Attention_weights_e)
    V1=redimension(dAttention_weights_e@K_E/cp.sqrt(dk))  
    V2=redimension(V_E) 
    V3=V1*V2 
    dLoss_Q_E=dLoss_Ae*V3  
    dLoss_inpute_q=dLoss_Q_E@Qe
    #print("dLoss_inpute_q.shape",dLoss_inpute_q.shape)
    
    V1=redimension(dAttention_weights_e@Q_E/cp.sqrt(dk))  
    V2=redimension(V_E) 
    V3=V1*V2 
    dLoss_K_E=dLoss_Ae*V3  
    dLoss_inpute_k=dLoss_K_E@Ke
    #print("dLoss_inpute_k.shape",dLoss_inpute_k.shape)
    dLoss_inpute=dLoss_inpute_a+dLoss_inpute_k+dLoss_inpute_q+dLoss_inpute_v
    dLoss_dWemb_encoder=dLoss_inpute*inputs_e
    return dLoss_inpute,dLoss_dWemb_encoder





## Update

In [7]:
def print_vocabs(ans,vocabulary): 
    for idx, values in enumerate(ans):
        max_index = cp.argmax(values)
        
        # Step 2: Find the word in the vocabulary with the corresponding position
        matched_word = None
        for word, (_, position) in vocabulary.items():
            if position == max_index:
                matched_word = word
                break
        print(f"List {idx + 1}: Max value index: {max_index}, Matched word: {matched_word}")

In [8]:
# Open the file with a specific encoding
with open('corpus/train.json', 'r', encoding='utf-8') as f:
    try:
        dataset = json.load(f)  # Load the JSON data
     
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")

X_train=["i love soy sauce!", 
         "my dog... is cute", 
         "you are crazy strong!",
         "the friend is good, you know"]

y_train=["io amo la salsa di soia!",
         "sei pazzo potente!",
        "il cane... è tenero",
        "l' arancio è buono, vero?"
        ]    



X_train = []
y_train = []

# #Loop through the list and process each dialogue and summary
for data in dataset:
    dialogue = data['dialogue']  # Split dialogue into a list of lines
    summary = data['summary']
    
    X_train.append(dialogue)
    y_train.append(summary)

nlp_encoder = spacy.load('en_core_web_lg')
nlp_decoder = spacy.load('it_core_news_lg')
# Disable unnecessary pipeline components
nlp_encoder.disable_pipes(["parser", "ner"])
nlp_decoder.disable_pipes(["parser", "ner"])


['parser', 'ner']

In [9]:
# batch_size=10
# #batch_size = len(X_train)
# words_per_phrase = 50
# dk = dv = embedding_size = 300 # constrain of transformer all embedding size both input embedding and attention embedding are the same encoder
# num_heads=10

# nlp_encoder = spacy.load('en_core_web_lg')
# nlp_decoder = spacy.load('it_core_news_lg')
# # Disable unnecessary pipeline components
# nlp_encoder.disable_pipes(["parser", "ner"])
# nlp_decoder.disable_pipes(["parser", "ner"])

# complete_text_target = ' '.join(y_train)
# vocabulary_decoder=create_vocabulary(complete_text_target+" [START] [PAD] [END] ","vocabulary_decoder",nlp_decoder) 
# complete_text_origin = ' '.join(X_train)
# vocabulary_encoder=create_vocabulary(complete_text_origin+" [START] [PAD] [END] ","vocabulary_encoder",nlp_encoder) 



# #inputs_e=create_input_encoder(X_train,nlp_encoder,words_per_phrase,embedding_size)
# #inputs_d=create_decoder_input(y_train,embedding_size,words_per_phrase,nlp_decoder)

# #inputs_e.shape,inputs_d.shape

In [10]:
nlp_encoder = spacy.load('en_core_web_lg')
nlp_decoder = spacy.load('it_core_news_lg')
# Disable unnecessary pipeline components
nlp_encoder.disable_pipes(["parser", "ner"])
nlp_decoder.disable_pipes(["parser", "ner"])

complete_text_target = ' '.join(y_train)
vocabulary_decoder=create_vocabulary(complete_text_target+" [START] [PAD] [END] ","vocabulary_decoder",nlp_decoder) 
complete_text_origin = ' '.join(X_train)
vocabulary_encoder=create_vocabulary(complete_text_origin+" [START] [PAD] [END] ","vocabulary_encoder",nlp_encoder) 

Function 'create_vocabulary' executed in 33.3205 seconds
Function 'create_vocabulary' executed in 46.3046 seconds


In [11]:
# batch_size=5
# num_batches_per_epoch = len(X_train) // batch_size

# #batch_size = len(X_train)
# words_per_phrase = 100
# dk = dv = embedding_size = 300 # constrain of transformer all embedding size both input embedding and attention embedding are the same encoder
# num_heads=10
# for i in tqdm(range(num_batches_per_epoch)):#desc=f"Epoch {epoch + 1}/{num_epochs}"
#     start = i * batch_size
#     end = start + batch_size
#     X_batch = X_train[start:end]
#     y_batch = y_train[start:end]
#     inputs_e=create_input_encoder(X_batch,vocabulary_encoder,words_per_phrase,embedding_size)
#     print("inputs_e",inputs_e.shape)
#     inputs_d=create_decoder_input(y_batch,embedding_size,words_per_phrase,vocabulary_decoder)
#     print("inputs_d",inputs_d.shape)
#     targets_d=create_target(y_batch,words_per_phrase,vocabulary_decoder)
#     print("targets_d",targets_d.shape)

In [12]:
# targets_d=create_target(y_train,words_per_phrase,vocabulary_decoder)
# targets_d.shape

In [13]:
learning_rate=0.01

vocab_size=len(vocabulary_decoder)
batch_size=5
num_batches_per_epoch = len(X_train) // batch_size

#batch_size = len(X_train)
words_per_phrase = num_phrases=50
dk = dv = embedding_size = 300 # constrain of transformer all embedding size both input embedding and attention embedding are the same encoder
num_heads=10
Qe = cp.random.rand(embedding_size, embedding_size) / cp.sqrt(embedding_size)
Ke = cp.random.rand(embedding_size, embedding_size) / cp.sqrt(embedding_size)
Ve = cp.random.rand(embedding_size, embedding_size) / cp.sqrt(embedding_size)
Qc = cp.random.rand(embedding_size, embedding_size) / cp.sqrt(embedding_size)
Kc = cp.random.rand(embedding_size, embedding_size) / cp.sqrt(embedding_size)
Vc = cp.random.rand(embedding_size, embedding_size) / cp.sqrt(embedding_size)
Qd = cp.random.rand(embedding_size, embedding_size) / cp.sqrt(embedding_size)
Kd = cp.random.rand(embedding_size, embedding_size) / cp.sqrt(embedding_size)
Vd = cp.random.rand(embedding_size, embedding_size) / cp.sqrt(embedding_size)

fl1_size=100
Wfl1e=cp.random.rand(embedding_size, fl1_size)   
bfl1e=cp.random.rand(fl1_size)

Wfl2e=cp.random.rand(fl1_size, dv)   
bfl2e=cp.random.rand(dv)


Wfl1d=cp.random.rand(embedding_size, fl1_size)    
bfl1d=cp.random.rand(fl1_size)

Wfl2d=cp.random.rand(fl1_size, dv)    
bfl2d=cp.random.rand(dv)

Wo=cp.random.rand((words_per_phrase)*embedding_size,vocab_size)   
bo=cp.random.rand(vocab_size)

In [14]:


for i in tqdm(range(num_batches_per_epoch)):#desc=f"Epoch {epoch + 1}/{num_epochs}"
    start = i * batch_size
    end = start + batch_size
    X_batch = X_train[start:end]
    y_batch = y_train[start:end]
    inputs_e=create_input_encoder(X_batch,vocabulary_encoder,words_per_phrase,embedding_size)
    print("inputs_e",inputs_e.shape)
    inputs_d=create_decoder_input(y_batch,embedding_size,words_per_phrase,vocabulary_decoder)
    print("inputs_d",inputs_d.shape)
    targets_d=create_target(y_batch,words_per_phrase,vocabulary_decoder)
    print("targets_d",targets_d.shape)

    Ae,Xe,Ect1,mu_e,var_e,Ne,Attention_weights_e,K_E,V_E,Q_E=forward_attention_encoder(inputs_e) 
    Ecout,mu_e2,var_e2,N_e2,FLe1,Xe1,Xe2=fully_connected_layers_encoder(Ect1)
    K_C,V_C=cross_attention_encoder(Ecout)

    step=0
    learning_rate=0.001
    for epochs in range(0,5):
        for step in range(inputs_d.shape[0]):
            inputs_decoder=inputs_d[step]
            target=targets_d[step]
            print(inputs_decoder.shape,target.shape)
            A_mask,Xd,Dt1,mu_d,var_d,N_d,Attention_weights_masked,Q_D,K_D,V_D=forward_attention_decoder(inputs_decoder)
            Q_C=cross_attention_decoder(Dt1)
            Dt2, mu_res,var_res,N_res,Res,Attention_weights_cross=cross_attention(Q_C,K_C,V_C,Dt1)
            Dout,mu_d2,var_d2,N_d2,Xd2,Xd1,FLd1=fully_connected_layers_decoder(Dt2)
            SigmaZout=output_layer(Dout)
            
            
            #print_vocabs(SigmaZout,vocabulary_decoder)
            Loss=loss_calculation(SigmaZout,target)
            print("step",step,"Loss",Loss)
            dLoss_Dout,dLoss_W0,dLoss_b0=derivate_dout(SigmaZout,target,Dout)
            dLoss_Wfl2d,dLoss_bfl2d,dLoss_Wfl1d,dLoss_bfl1d,DLoss_Dt2=derivate_fully_connected_layers_decoder(dLoss_Dout,Dt2,Xd2,var_d2,mu_d2,N_d2,Wfl2d,FLd1,Xd1)
            dLoss_Qc,dLoss_Kc,dLoss_Vc,Attention_weights_cross,dLoss_Dt1_a,dLoss_Acr=derivative_cross_attention(DLoss_Dt2,Res,var_res,mu_res,N_res,Attention_weights_cross,K_C,V_C,Q_C,Ecout,Dt1)
            dLoss_Kd,dLoss_Qd,dLoss_Vd,dLoss_inputd_a,dLoss_Amask=derivative_attention_decoder(dLoss_Acr,Attention_weights_cross,dLoss_Dt1_a,Attention_weights_masked,Q_D,V_D,K_D,K_C,V_C,Xd,var_d,mu_d,N_d,inputs_decoder)
            dLoss_inputd,dLoss_dWemb_decoder=derivative_input_decoder(dLoss_Amask,Attention_weights_masked,K_D,V_D,Q_D,dLoss_inputd_a,inputs_decoder)
            #print("dLoss_W0",dLoss_W0[0])
            Wo=Wo-learning_rate*dLoss_W0.T
            
            #print("W0",W0[0])
            bo=bo-learning_rate*dLoss_b0
            Wfl2d=Wfl2d-learning_rate*dLoss_Wfl2d.T
            bfl2d=bfl2d-learning_rate*dLoss_bfl2d
            Wfl1d=Wfl1d-learning_rate*dLoss_Wfl1d.T
            bfl1d=bfl1d-learning_rate*dLoss_bfl1d
            Qc=Qc-learning_rate*dLoss_Qc
            Kc=Kc-learning_rate*dLoss_Kc
            Vc=Vc-learning_rate*dLoss_Vc
            Qd=Qd-learning_rate*dLoss_Qd
            Kd=Kd-learning_rate*dLoss_Kd
            Vd=Vd-learning_rate*dLoss_Vd
            inputs_d=inputs_d-learning_rate*dLoss_dWemb_decoder
            #print(input_d)

        dLoss_Ecout=derivative_Ecout(Attention_weights_cross,dLoss_Acr,Q_C,V_C)
        dLoss_dWfl2e,dLoss_dbfl2e,dLoss_Wfl1e,dLoss_bfl1e,dLoss_Ect1=derivate_fully_connected_layers_encoder(dLoss_Ecout,Ect1,Xe2,var_e2,mu_e2,N_e2,FLe1,Xe1)
        dLoss_dQe,dLoss_dKe,dLoss_dVe,dLoss_inpute_a,dLoss_Ae=derivative_attention_encoder(dLoss_Ect1,Xe,var_e,mu_e,Ne,Attention_weights_e,K_E,V_E,Q_E)
        dLoss_inpute,dLoss_dWemb_encoder=derivative_input_encoder(dLoss_Ae,Attention_weights_e,K_E,V_E,Q_E,dLoss_inpute_a)
        Wfl2e=Wfl2e-learning_rate*cp.sum(cp.transpose(dLoss_dWfl2e ,(0,2,1)),axis=0) 
        bfl2e=bfl2e-learning_rate*bfl2e
        Wfl1e=Wfl1e-learning_rate*cp.sum(cp.transpose(dLoss_Wfl1e ,(0,2,1)),axis=0) 
        bfl1e=bfl1e-learning_rate*bfl1e
        Qe=Qe-learning_rate*dLoss_dQe
        Ke=Ke-learning_rate*dLoss_dKe
        Ve=Ve-learning_rate*dLoss_dVe
        #inputs_e=inputs_e-learning_rate*dLoss_dWemb_encoder

  0%|          | 0/2946 [00:00<?, ?it/s]

Function 'pad_sequences' executed in 0.0000 seconds
Function 'generate_input_encoder' executed in 0.0012 seconds
Function 'create_input_encoder' executed in 0.3282 seconds
inputs_e (5, 50, 300)
Function 'pad_sequences' executed in 0.0000 seconds
Function 'create_timestaped_input' executed in 0.0206 seconds
Function 'create_decoder_input' executed in 0.0226 seconds
inputs_d (50, 5, 50, 300)
Function 'pad_sequences' executed in 0.0000 seconds
Function 'create_timestaped_input' executed in 0.1383 seconds
Function 'create_target' executed in 0.1557 seconds
targets_d (50, 5, 17533)
(5, 50, 300) (5, 17533)
step 0 Loss 148672.85376742092
(5, 50, 300) (5, 17533)
step 1 Loss 196875.1701041583
(5, 50, 300) (5, 17533)
step 2 Loss 151598.97890359777
(5, 50, 300) (5, 17533)
step 3 Loss 204886.78467672179
(5, 50, 300) (5, 17533)
step 4 Loss 183943.8522219037
(5, 50, 300) (5, 17533)
step 5 Loss 194413.24612272906
(5, 50, 300) (5, 17533)
step 6 Loss 106583.9008525828
(5, 50, 300) (5, 17533)
step 7 Los

  0%|          | 0/2946 [02:37<?, ?it/s]


KeyboardInterrupt: 

In [12]:
#vocabulary["[PAD]"]